In [1]:
#!/usr/bin/env python
# coding: utf-8

import os
import sys
import platform
import logging
import argparse
from datetime import datetime, date, timedelta
import pendulum
import pandas as pd
import numpy as np
import re
from IPython.display import display

In [2]:
# Add parent directory to path
repo_path = '/home/notebook/prod/nelson/dview-naarni-data-platform'
sys.path.append(os.path.join(repo_path, 'tasks'))

# Import necessary files and its respective functions
from common.db_operations import connect_to_trino, fetch_data_for_day, write_df_to_iceberg,drop_table,execute_query
from common.optimizer_logic import optimize_dataframe_memory

# Import business logic functions
from biz_logic.energy_mileage.energy_mileage_daily_v0 import energy_mileage_stats ,impute_odometer_readings

from biz_logic.energy_consumption.energy_consumption_report import energy_consumption_stats

# Configure basic logging
logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s - %(levelname)s - %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S')

# Print the Python version being used
print(f"Using Python version: {platform.python_version()}")

Using Python version: 3.11.13


In [3]:
def parse_copy_blocks(sql_path):
    """
    Parses a PostgreSQL dump file and extracts COPY blocks into:
    {table_name: {"columns": [...], "rows": [...]}}
    """

    with open(sql_path, "r", encoding="utf-8") as f:
        lines = f.readlines()

    copy_pattern = re.compile(r'^COPY\s+public\.([a-zA-Z0-9_]+)\s*\((.*?)\)\s+FROM\s+stdin;')
    
    tables = {}
    current_table = None

    for line in lines:
        match = copy_pattern.match(line)
        if match:
            table_name = match.group(1)
            columns = [c.strip() for c in match.group(2).split(",")]

            current_table = table_name
            tables[current_table] = {"columns": columns, "rows": []}
            continue

        if current_table:
            if line.strip() == r"\.":
                current_table = None
            else:
                # Split COPY row by tabs
                row = [None if v == r"\N" else v for v in line.rstrip("\n").split("\t")]
                tables[current_table]["rows"].append(row)

    return tables


In [4]:
# ---- CLI Argument Parser ----
def is_running_in_jupyter():
    try:
        from IPython import get_ipython
        if 'IPKernelApp' in get_ipython().config:
            return True
    except Exception:
        pass
    return False


def parse_arguments():
    parser = argparse.ArgumentParser(description="Process facts-prod bizlogic data.")
    parser.add_argument("--date", type=str, help="Date to process in YYYY-MM-DD format (IST).")
    parser.add_argument("--yesterday", action="store_true", help="Process yesterday's data (overrides --date if both are provided)")

    if is_running_in_jupyter():
        class DefaultArgs:
            date = None
            yesterday = False
        return DefaultArgs()

    return parser.parse_args()


# ---- Date Handling (Fixed Timezone Logic) ----
def get_target_date(args):
    """Determine which date's data to process (IST-based)."""
    tz = pendulum.timezone("Asia/Kolkata")

    if args.date:
        target_date = pendulum.parse(args.date, tz=tz).date()
    elif args.yesterday:
        target_date = pendulum.now(tz).subtract(days=1).date()
    else:
        target_date = pendulum.now(tz).date()

    logging.info(f"Script running at {pendulum.now(tz)} (IST)")
    logging.info(f"Extracting data for {target_date}")
    return target_date


def process_data(conn, df: pd.DataFrame, table_name: str, schema: str = "facts_prod", push_to_db: bool = True, partition_by: list = None):
    # Process data related energy-mileage relation
    df_res = df
    if not df_res.empty:
        if push_to_db:
            write_df_to_iceberg(conn, df_res, table_name, schema, partition_by=partition_by)
            logging.info(f"✅ Processing and write for {table_name} complete with partitioning by {partition_by}.")
        else:
            logging.info(f"✅ Processing for {table_name} complete. Data not pushed to database.")
    else:
        logging.info("Processed DataFrame is empty. No data to write.")
    return df_res

In [5]:
sql_path = "naarni_backend.sql"   # Path to your file
tables_data = parse_copy_blocks(sql_path)

print("Tables detected:", list(tables_data.keys()))

Tables detected: ['cities', 'device_vehicle_mappings', 'fleet_operator_mappings', 'fleets', 'organizations', 'routes', 'user_accounts', 'user_profiles', 'vehicle_devices', 'vehicle_goals', 'vehicles']


In [6]:
def try_parse_datetime(series):
    """
    Attempt fast datetime conversion.
    Avoid pandas format warnings by suppressing inference alerts.
    If parsing fails, return original series unchanged.
    """

    # Skip non-string columns
    if series.dtype != object:
        return series

    # Try to detect datetime-looking strings (cheap check)
    sample = series.dropna().astype(str).head(5)
    if not any(c in s for s in sample for c in ["-", ":", "T", "+"]):
        return series  # probably not datetime

    try:
        # errors='coerce' converts bad timestamps to NaT
        converted = pd.to_datetime(series, errors="coerce", utc=True)
        
        # If too many coercions happened, abort conversion
        non_null_ratio = converted.notna().mean()
        if non_null_ratio < 0.5:  # less than 50% valid timestamps → treat as non-datetime column
            return series
        
        return converted

    except Exception:
        return series


In [7]:
conn = connect_to_trino()

target_schema = "facts_dev"

for table, info in tables_data.items():
    print(f"\n🚀 Processing table: {table}")
    cols = info["columns"]
    rows = info["rows"]

    df = pd.DataFrame(rows, columns=cols)

    # Convert numeric-looking columns
    for c in df.columns:
        # Try float
        try:
            df[c] = pd.to_numeric(df[c])
            continue
        except:
            pass
        # Try datetime
        try:
            df[c] = try_parse_datetime(df[c])
        except:
            pass

    print(f"📊 Rows loaded for {table}: {len(df)}")

    # Write to Iceberg (Trino)
    write_df_to_iceberg(conn, df, table=table, schema=target_schema)

print("\n🎉 ALL TABLES IMPORTED SUCCESSFULLY INTO ICEBERG 🎉")


2025-12-04 19:52:38 - INFO - 🔌 STEP 1: Connecting to Trino...
2025-12-04 19:52:38 - INFO - ✅ STEP 1: Connected to Trino
/tmp/ipykernel_2547/2163252617.py:19: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  converted = pd.to_datetime(series, errors="coerce", utc=True)
2025-12-04 19:52:38 - INFO - 💾 [4/5] STEP 4a: Preparing to write results to Iceberg table adhoc.facts_dev.cities...
2025-12-04 19:52:38 - INFO - 🔍 [4/5] STEP 4b: Creating table if not exists...



🚀 Processing table: cities
📊 Rows loaded for cities: 179


2025-12-04 19:52:38 - INFO - 🛠️ [4/5] STEP 4c: Ensured Iceberg table cities exists
2025-12-04 19:52:38 - INFO - 🔍 [4/5] STEP 4d: Prepared INSERT statement
2025-12-04 19:52:38 - INFO - 🔍 [4/5] STEP 4e: First row preview: (1, datetime.datetime(2025, 11, 18, 8, 40, 1, 583805), datetime.datetime(2025, 11, 18, 8, 40, 1, 583805), 'New Delhi', 'Delhi', 'India', 28.61, 77.21, 't', 'National Capital of India')
2025-12-04 19:54:34 - INFO - ✅ [4/5] STEP 4f: Inserted 179 rows (total 179) into cities
2025-12-04 19:54:34 - INFO - 🎉 [4/5] STEP 4g: Finished inserting 179 rows into cities
2025-12-04 19:54:34 - INFO - 💾 [4/5] STEP 4a: Preparing to write results to Iceberg table adhoc.facts_dev.device_vehicle_mappings...
2025-12-04 19:54:34 - INFO - 🔍 [4/5] STEP 4b: Creating table if not exists...
2025-12-04 19:54:34 - INFO - 🛠️ [4/5] STEP 4c: Ensured Iceberg table device_vehicle_mappings exists
2025-12-04 19:54:34 - INFO - 🔍 [4/5] STEP 4d: Prepared INSERT statement
2025-12-04 19:54:34 - INFO - 🔍 [4/5] S


🚀 Processing table: device_vehicle_mappings
📊 Rows loaded for device_vehicle_mappings: 25


2025-12-04 19:54:46 - INFO - ✅ [4/5] STEP 4f: Inserted 25 rows (total 25) into device_vehicle_mappings
2025-12-04 19:54:46 - INFO - 🎉 [4/5] STEP 4g: Finished inserting 25 rows into device_vehicle_mappings
2025-12-04 19:54:46 - INFO - 💾 [4/5] STEP 4a: Preparing to write results to Iceberg table adhoc.facts_dev.fleet_operator_mappings...
2025-12-04 19:54:46 - INFO - 🔍 [4/5] STEP 4b: Creating table if not exists...
2025-12-04 19:54:46 - INFO - 🛠️ [4/5] STEP 4c: Ensured Iceberg table fleet_operator_mappings exists
2025-12-04 19:54:46 - INFO - 🔍 [4/5] STEP 4d: Prepared INSERT statement
2025-12-04 19:54:46 - INFO - 🔍 [4/5] STEP 4e: First row preview: (4, datetime.datetime(2025, 11, 20, 8, 0, 28, 783537), datetime.datetime(2025, 11, 20, 8, 0, 28, 783537), 1, 2, datetime.datetime(2025, 7, 1, 0, 0), None, None, None, None, None)


...


🚀 Processing table: vehicle_goals
📊 Rows loaded for vehicle_goals: 44


2025-12-04 19:55:16 - INFO - 🛠️ [4/5] STEP 4c: Ensured Iceberg table vehicle_goals exists
2025-12-04 19:55:16 - INFO - 🔍 [4/5] STEP 4d: Prepared INSERT statement
2025-12-04 19:55:16 - INFO - 🔍 [4/5] STEP 4e: First row preview: (1, datetime.datetime(2025, 11, 19, 8, 36, 34, 624608), datetime.datetime(2025, 11, 19, 8, 36, 34, 624608), 1, 1, 1, 500, 't', datetime.datetime(2025, 11, 19, 8, 36, 34, 623739), None, None, None)
2025-12-04 19:55:40 - INFO - ✅ [4/5] STEP 4f: Inserted 44 rows (total 44) into vehicle_goals
2025-12-04 19:55:40 - INFO - 🎉 [4/5] STEP 4g: Finished inserting 44 rows into vehicle_goals
/tmp/ipykernel_2547/2163252617.py:19: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  converted = pd.to_datetime(series, errors="coerce", utc=True)
/tmp/ipykernel_2547/2163252617.py:19: UserWarning: Could not infer format, so each element will be parsed


🚀 Processing table: vehicles
📊 Rows loaded for vehicles: 25


2025-12-04 19:55:54 - INFO - ✅ [4/5] STEP 4f: Inserted 25 rows (total 25) into vehicles
2025-12-04 19:55:54 - INFO - 🎉 [4/5] STEP 4g: Finished inserting 25 rows into vehicles



🎉 ALL TABLES IMPORTED SUCCESSFULLY INTO ICEBERG 🎉


In [8]:
for t in tables_data.keys():
    df_check = execute_query(conn, f"SELECT COUNT(*) AS rows FROM adhoc.facts_dev.{t}")
    print(t, df_check)


2025-12-04 19:55:54 - INFO - ⚙️ Executing query...
2025-12-04 19:55:55 - INFO - ✅ Query executed successfully with results!
2025-12-04 19:55:55 - INFO - ⚙️ Executing query...


cities    rows
0   179


2025-12-04 19:55:55 - INFO - ✅ Query executed successfully with results!
2025-12-04 19:55:55 - INFO - ⚙️ Executing query...
2025-12-04 19:55:55 - INFO - ✅ Query executed successfully with results!
2025-12-04 19:55:55 - INFO - ⚙️ Executing query...
2025-12-04 19:55:55 - INFO - ✅ Query executed successfully with results!
2025-12-04 19:55:55 - INFO - ⚙️ Executing query...


device_vehicle_mappings    rows
0    25
fleet_operator_mappings    rows
0     4
fleets    rows
0     2


2025-12-04 19:55:55 - INFO - ✅ Query executed successfully with results!
2025-12-04 19:55:55 - INFO - ⚙️ Executing query...
2025-12-04 19:55:55 - INFO - ✅ Query executed successfully with results!
2025-12-04 19:55:55 - INFO - ⚙️ Executing query...


...

vehicle_devices    rows
0    25


2025-12-04 19:55:56 - INFO - ✅ Query executed successfully with results!
2025-12-04 19:55:56 - INFO - ⚙️ Executing query...


vehicle_goals    rows
0    44


2025-12-04 19:55:57 - INFO - ✅ Query executed successfully with results!


vehicles    rows
0    25


In [9]:
for t in tables_data.keys():
    df_check = execute_query(conn, f"SELECT * FROM adhoc.facts_dev.{t} limit 5")
    display(t, df_check)


2025-12-04 19:55:57 - INFO - ⚙️ Executing query...
2025-12-04 19:55:57 - INFO - ✅ Query executed successfully with results!


'cities'

,id,created_at,last_modified_at,name,state,country,latitude,longitude,is_active,description
0,131,2025-11-18 08:40:01.583805,2025-11-18 08:40:01.583805,Goalpara,Assam,India,26.17,90.63,t,Agricultural Hub
1,126,2025-11-18 08:40:01.583805,2025-11-18 08:40:01.583805,Jorhat,Assam,India,26.75,94.20,t,Cultural Capital
2,146,2025-11-18 08:40:01.583805,2025-11-18 08:40:01.583805,Korba,Chhattisgarh,India,22.35,82.68,t,Power Hub
3,105,2025-11-18 08:40:01.583805,2025-11-18 08:40:01.583805,Siliguri,West Bengal,India,26.73,88.40,t,Gateway to Northeast
4,133,2025-11-18 08:40:01.583805,2025-11-18 08:40:01.583805,Bhopal,Madhya Pradesh,India,23.26,77.41,t,City of Lakes


2025-12-04 19:55:57 - INFO - ⚙️ Executing query...
2025-12-04 19:55:57 - INFO - ✅ Query executed successfully with results!


'device_vehicle_mappings'

...

'vehicle_goals'

,id,created_at,last_modified_at,vehicle_id,route_id,day,target_km_run,is_active,effective_from,effective_until,notes,created_by
0,13,2025-11-26 04:15:05.664708,2025-11-26 04:15:05.664708,3,4,5,800,t,2025-11-26 04:15:05.664564,None,None,None
1,9,2025-11-26 04:14:29.664971,2025-11-26 04:14:29.664971,3,4,2,800,t,2025-11-26 04:14:29.664829,None,None,None
2,14,2025-11-26 04:15:13.331407,2025-11-26 04:15:13.331407,3,4,6,800,t,2025-11-26 04:15:13.331233,None,None,None
3,36,2025-11-26 04:18:54.194055,2025-11-26 04:18:54.194055,7,3,7,700,t,2025-11-26 04:18:54.193889,None,None,None
4,44,2025-11-26 04:20:02.244604,2025-11-26 04:20:02.244604,8,3,7,700,t,2025-11-26 04:20:02.244452,None,None,None


2025-12-04 19:55:57 - INFO - ⚙️ Executing query...
2025-12-04 19:55:57 - INFO - ✅ Query executed successfully with results!


'vehicles'

,id,created_at,last_modified_at,registration_number,model,make,year,capacity,status,is_active,fleet_id
0,10,2025-11-20 06:27:14.804087,2025-11-20 06:27:14.804087,TN14AR3907,AZAD 12.5 M Luxury intercity AC Coach - seater...,AZAD,2025,None,NOT_MOVING,t,2
1,7,2025-11-20 06:23:05.082257,2025-11-20 06:23:59.068766,AP39WG4628,A_12_M,AZAD,2025,None,NOT_MOVING,t,2
2,6,2025-11-20 06:22:05.430863,2025-11-20 06:22:05.430863,AP39WG0271,AZAD 12.5 M Luxury intercity AC Coach - seater...,AZAD,2025,None,NOT_MOVING,t,2
3,25,2025-11-20 06:55:44.330442,2025-11-20 06:55:44.330442,DL1PD8509,AZAD 12.5 M Luxury intercity AC Coach - seater...,AZAD,2025,None,NOT_MOVING,t,1
4,14,2025-11-20 06:31:10.998892,2025-11-20 06:31:10.998892,TN14AR7552,AZAD 12.5 M Luxury intercity AC Coach - seater...,AZAD,2025,None,NOT_MOVING,t,2


In [10]:
wait

NameError: name 'wait' is not defined

In [ ]:
conn = connect_to_trino()

tables = [
    "cities", "device_vehicle_mappings", "fleet_operator_mappings",
    "fleets", "organizations", "routes", "user_accounts",
    "user_profiles", "vehicle_devices", "vehicles", "vehicle_goals"
]

for t in tables:
    print(f"Replicating {t} → facts_prod...")
    sql = f"""
        CREATE TABLE IF NOT EXISTS adhoc.facts_prod.{t} AS
        SELECT * FROM adhoc.facts_dev.{t}
    """
    execute_query(conn, sql, return_results=False)

print("🎉 All tables replicated successfully!")
conn.close()

In [ ]:
conn = connect_to_trino()

tables_to_drop = [
    "cities",
    "device_vehicle_mappings",
    "fleet_operator_mappings",
    "fleets",
    "organizations",
    "routes",
    "user_accounts",
    "user_profiles",
    "vehicle_devices",
    "vehicles",
    "vehicle_goals",
]

for t in tables_to_drop:
    sql = f"DROP TABLE IF EXISTS adhoc.facts_dev.{t}"
    print(f"Dropping ONLY: {t}")
    execute_query(conn, sql, return_results=False)

print("✅ Safe cleanup complete. Only the intended tables were removed.")
conn.close()